In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
sns.set_style('darkgrid')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.2f}'.format
import nltk

In [2]:
import string
punctuations = string.punctuation
from nltk.corpus import stopwords
stopword_list = stopwords.words("english")
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
def clean(text):
    cleaned_text = text.lower()
    cleaned_text=re.sub("[^a-zA-Z]"," ",cleaned_text)
    cleaned_text=re.sub(r'\b\w{1,3}\b', '',cleaned_text)

    cleaned_text = "".join(c for c in cleaned_text if c not in punctuations)
    words = cleaned_text.split()
    words = [w for w in words if w not in stopword_list]
     
    words = [lem.lemmatize(word,"v") for word in words]
    words = [lem.lemmatize(word,"n") for word in words]
    words = [lem.lemmatize(word,"r") for word in words]
    cleaned_text = " ".join(words)
    
    return cleaned_text

In [3]:
df=pd.read_csv('tweetdata.csv')
df.drop(columns =['id', 'Unnamed: 0', 'score'],axis=1, inplace=True)
df.head()

,tweet,label
0,At the point today where if someone says somet...,anger
1,@CorningFootball IT'S GAME DAY!!!! T MIN...,anger
2,This game has pissed me off more than any othe...,anger
3,@spamvicious I've just found out it's Candice ...,anger
4,@moocowward @mrsajhargreaves @Melly77 @GaryBar...,anger


In [5]:
from nltk.corpus import stopwords
import itertools
import time

In [6]:
start_time = time.time()

In [8]:
stopset = set(stopwords.words('english'))

In [9]:
from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()

In [10]:
def cleaning(text):
    txt = str(text)
    txt = re.sub(r"http\S+", "", txt)
    if len(txt) == 0:
        return 'no text'
    else:
        txt = txt.split()
        index = 0
        for j in range(len(txt)):
            if txt[j][0] == '@':
                index = j
        txt = np.delete(txt, index)
        if len(txt) == 0:
            return 'no text'
        else:
            words = txt[0]
            for k in range(len(txt)-1):
                words+= " " + txt[k+1]
            txt = words
            txt = re.sub(r'[^\w]', ' ', txt)
            if len(txt) == 0:
                return 'no text'
            else:
                txt = ''.join(''.join(s)[:2] for _, s in itertools.groupby(txt))
                txt = txt.replace("'", "")
                txt = nltk.tokenize.word_tokenize(txt)
                #data.content[i] = [w for w in data.content[i] if not w in stopset]
                for j in range(len(txt)):
                    txt[j] = lem.lemmatize(txt[j], "v")
                if len(txt) == 0:
                    return 'no text'
                else:
                    return txt

In [12]:
df['clean_text'] = df['clean_text'].map(lambda x: cleaning(x))
        
df = df.reset_index(drop=True)
for i in range(len(df)):
    words = df.clean_text[i][0]
    for j in range(len(df.clean_text[i])-1):
        words+= ' ' + df.clean_text[i][j+1]
    df.clean_text[i] = words

In [13]:
x = int(np.round(len(df)*0.75))
train = df.iloc[:x,:].reset_index(drop = True)
test = df.iloc[x:,:].reset_index(drop = True)

In [15]:
!pip install textblob

In [16]:
from textblob.classifiers import NaiveBayesClassifier as NBC

In [18]:
training_corpus = []

for k in range(len(train)):
    training_corpus.append((train.clean_text[k], train.label[k]))    
test_corpus = []

for l in range(len(test)):
    test_corpus.append((test.clean_text[l], test.label[l]))

model = NBC(training_corpus)

print(model.accuracy(test_corpus))

0.540785498489426


In [20]:
from sklearn.metrics import classification_report

predictions = []
for m in range(len(test)):
    predictions.append(model.classify(test.clean_text[m]))
print(classification_report(test.label, predictions))
    
predictions_df = pd.DataFrame({'clean_text':test.clean_text, 'Emotion_predicted':predictions, 'Emotion_actual':test.label})
predictions_df.to_csv('naive_emotion_recognizer.csv', index = False)

elapsed_time = time.time() - start_time
print ("processing time:", elapsed_time, "seconds")

              precision    recall  f1-score   support

       anger       0.00      0.00      0.00         0
        fear       0.96      0.15      0.25      1039
         joy       0.55      0.83      0.66       823
     sadness       0.53      0.76      0.63       786

    accuracy                           0.54      2648
   macro avg       0.51      0.43      0.38      2648
weighted avg       0.70      0.54      0.49      2648

processing time: 4515.350930452347 seconds


In [21]:
predictions_df

,clean_text,Emotion_predicted,Emotion_actual
0,go submit wardrobe makeover show fear fashion,joy,fear
1,social medium another father take child nightm...,joy,fear
2,shaun,joy,fear
3,time stress always year age make redundant start,joy,fear
4,steal tell,joy,fear
...,...,...,...
2643,much home stun happy think sink,sadness,sadness
2644,n o t e x t,anger,sadness
2645,pretty love background purple highlight dull c...,sadness,sadness
2646,artist announcement look good bluesfest blue m...,sadness,sadness
